# Oil Consumption Multiple Regression Machine Learning

In [1]:
# Import dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [2]:
# Read the csv files into a pandas DataFrame

oil_prod = pd.read_csv('../data/clean_data/Oil Consumption - Barrels-YearFixed-Python.csv')
oil_prod = oil_prod.rename(columns={"Total World": "World Barrels"})
pop = pd.read_csv('../data/clean_data/WorldPopulationbyYear.csv')
pop = pop.rename(columns={"World": "World Population"})
oil_pop = pop.merge(oil_prod, on="Year")
gdp = pd.read_csv('../data/clean_data/GDP%-YearFixed-Python.csv', encoding = "ISO-8859-1")
filteredGDP = gdp[["Year", "United States", "World"]]
filteredGDP = filteredGDP.rename(columns={"United States": "US GDP%", "World": "World GDP%"})
inflation = pd.read_csv('../data/clean_data/InflationAnnual%-YearFixed-Python.csv', encoding = "ISO-8859-1")
filteredInflation = inflation[["Year", "United States", "World"]]
filteredInflation = filteredInflation.rename(columns={"United States": "US Inflation%", "World": "World Inflation%"})
goodsTax = pd.read_csv('../data/clean_data/Taxes%-YearFixed-Python.csv', encoding = "ISO-8859-1")
filteredGoodsTax = goodsTax[["Year", "United States", "World"]]
filteredGoodsTax = filteredGoodsTax.rename(columns={"United States": "US Goods Tax%", "World": "World Goods Tax%"})

# Merge dataframes on Year
gdp_oil_pop = oil_pop.merge(filteredGDP, on="Year")
gdp_oil_pop_infl = gdp_oil_pop.merge(filteredInflation, on="Year")
full_merge = gdp_oil_pop_infl.merge(filteredGoodsTax, on="Year")

# Drop 2019 row - There's multiple features set to 0.  Will later run prediction on 2019
full_merge = full_merge.drop([54])

# Clean NaNs - change to 0 so columns are same length
full_merge = full_merge.replace(np.nan, 0)
full_merge

,Year,World Population,Algeria,Argentina,Australia,Austria,Azerbaijan,Bangladesh,Belarus,Belgium,...,Uzbekistan,Venezuela,Vietnam,Western Africa,US GDP%,World GDP%,US Inflation%,World Inflation%,US Goods Tax%,World Goods Tax%
0,1965,3322973367,26.716192,432.373936,312.980082,106.652849,0.000000,0.000000,0.000000,312.112712,...,0.000000,183.879953,31.347253,74.367779,0.000000,0.000000,1.585169,0.000000,0.000000,0.000000
1,1966,3393031801,35.353233,447.270806,367.427507,118.205781,0.000000,0.000000,0.000000,318.668110,...,0.000000,181.469394,68.509536,80.171376,0.000000,0.000000,3.015075,0.000000,0.000000,0.000000
2,1967,3462460201,33.285973,459.876977,401.463178,125.648575,0.000000,0.000000,0.000000,350.395123,...,0.000000,185.728871,100.338166,81.510241,0.000000,0.000000,2.772786,0.000000,0.000000,0.000000
3,1968,3532826854,35.374426,468.994810,435.991175,144.321038,0.000000,0.000000,0.000000,405.103388,...,0.000000,201.425590,102.080276,81.265671,0.000000,0.000000,4.271796,0.000000,0.000000,0.000000
4,1969,3607499991,37.714329,491.750733,450.960822,158.686931,0.000000,0.000000,0.000000,468.287863,...,0.000000,200.370616,123.713820,89.834801,0.000000,0.000000,5.462386,0.000000,0.000000,0.000000
5,1970,3682911039,43.009918,447.785768,496.753507,175.707808,0.000000,0.000000,0.000000,511.709370,...,0.000000,209.874364,128.258016,98.757641,21.414736,26.911121,5.838255,0.000000,0.000000,0.000000
6,1971,3760509002,48.878493,479.603260,523.101014,196.213233,0.000000,14.006055,0.000000,525.207863,...,0.000000,212.341753,108.502315,129.464663,21.919818,26.526697,4.292767,0.000000,0.000000,0.000000
7,1972,3836892580,53.543497,478.584153,532.159372,211.859454,0.000000,16.867568,0.000000,566.885410,...,0.000000,230.821601,111.719328,138.260714,22.580622,26.164536,3.272278,0.000000,7.143859,0.000000
8,1973,3912347640,58.849260,482.634685,569.900630,230.396247,0.000000,18.974685,0.000000,597.895507,...,0.000000,257.258740,109.934923,152.970148,23.331809,27.054172,6.177760,0.000000,6.579487,0.000000
9,1974,3988478324,65.641863,482.874384,603.855644,207.130712,0.000000,19.626384,0.000000,529.360055,...,0.000000,259.424603,67.510698,158.582745,22.694942,27.896165,11.054805,0.000000,5.990202,0.000000


## Linear Regression

In [3]:
model = LinearRegression()

## One-step Forecast

In [4]:
# Using 2000 - 2009 data to run historical prediction 2001 - 2010

predict0110=[]

for year in range(10):
    # Starting on row 36 (2001)
    i = 36 + year

    # Does not need .value.reshape(-1, 1) as there's dimension now with 2+ features
    hist_X = full_merge[["World Population", "World Inflation%", "World Goods Tax%", "World GDP%"]]
    hist_y = full_merge["World Barrels"].values.reshape(-1, 1)
    X_scaler = StandardScaler().fit(hist_X)
    y_scaler = StandardScaler().fit(hist_y)
    X_train_scaled = X_scaler.transform(hist_X)
    y_train_scaled = y_scaler.transform(hist_y)
    X_train_scaled = pd.DataFrame(X_train_scaled)
    fitment = model.fit(X_train_scaled.iloc[(i-21):i], y_train_scaled[(i-21):i])
    
    # changed reshape to (1, -1)
    oil_predict = fitment.predict(X_train_scaled.iloc[i-1].values.reshape(1, -1))
    predict0110.append(oil_predict.flatten()[0])
    
# Invert predict0110 so it's not scaled for later comoparison
inv_predict0110 = y_scaler.inverse_transform(predict0110)

print(inv_predict0110)

[75812.04528625 77029.90082419 78089.09413699 79569.36767161
 81969.3024811  83524.56705058 85138.70249599 86512.45821575
 85690.39899609 85489.3968121 ]


## Historical Prediction MSE and R-Square

In [5]:
# Use our model to make predictions

predicted = model.predict(X_train_scaled)

# inv_predicted = y_scaler.inverse_transform(predicted)

hist_mse = mean_squared_error(y_train_scaled, predicted)
hist_r2 = r2_score(y_train_scaled, predicted)

print(f"Mean Squared Error (MSE): {hist_mse}")
print(f"R-squared (R2): {hist_r2}")

Mean Squared Error (MSE): 0.2895465064487502
R-squared (R2): 0.7104534935512499


## Historical Predictions

In [6]:
# Generate Historical Prediction table with difference to actual numbers

hist_pred_0110_df = full_merge.loc[full_merge['Year'].between(2001, 2010), ['Year', 'World Barrels']]
hist_pred_0110_df["Prediction"] = inv_predict0110
hist_pred_0110_df["Difference"] = hist_pred_0110_df["Prediction"] - hist_pred_0110_df["World Barrels"]
hist_pred_0110_df["% Difference"] = ((hist_pred_0110_df["Prediction"] - hist_pred_0110_df["World Barrels"])/hist_pred_0110_df["World Barrels"])*100
hist_pred_0110_df

,Year,World Barrels,Prediction,Difference,% Difference
36,2001,77365.72931,75812.045286,-1553.684024,-2.008233
37,2002,78238.28865,77029.900824,-1208.387826,-1.544497
38,2003,79907.62267,78089.094137,-1818.528533,-2.275789
39,2004,82654.46174,79569.367672,-3085.094068,-3.732520
40,2005,83891.17801,81969.302481,-1921.875529,-2.290915
41,2006,84915.99774,83524.567051,-1391.430689,-1.638597
42,2007,86099.63390,85138.702496,-960.931404,-1.116069
43,2008,85170.13717,86512.458216,1342.321046,1.576047
44,2009,84082.70720,85690.398996,1607.691796,1.912036
45,2010,86855.60512,85489.396812,-1366.208308,-1.572965


## Save Historical Predictions to CSV

In [7]:
# Export Historical Predictions table as CSV

hist_pred_0110_df.to_csv('../data/clean_data/oil_outputs/OilConsumption_Historical_MultipleRegression_2001_2010.csv', index=False)

## Features' Rolling Average for 2019 - 2023

In [8]:
# Narrow down data frame to the specific year range of 2010 - 2018

multi_feat = full_merge.loc[full_merge['Year'].between(2010, 2018), ['Year',
                                                                     'World Population', 
                                                                   'World Barrels', 
                                                                   'World Inflation%', 
                                                                   'World Goods Tax%', 
                                                                   'World GDP%']]
multi_feat

,Year,World Population,World Barrels,World Inflation%,World Goods Tax%,World GDP%
45,2010,6921871614,86855.60512,3.326345,31.875890,24.207113
46,2011,7002860604,87820.24074,4.839403,33.264196,24.547417
47,2012,7085763408,88784.26340,3.707818,33.271756,24.404915
48,2013,7169640142,90151.54476,2.605818,32.787076,24.310278
49,2014,7254228377,90902.90771,2.346269,33.191709,24.470283
50,2015,7338964960,92610.12861,1.393330,33.724915,24.297531
51,2016,7424282488,94403.97687,1.486007,34.248831,23.913640
52,2017,7509065705,96012.59157,2.233522,33.333664,24.222791
53,2018,7591932907,97348.38002,2.458142,34.011405,24.382773


In [9]:
# Iterate 5 times for 5 years (2019 - 2023) of rolling average of features

for i in range(5):
    starting_index = 4 + i
    year_inc = 2018 + i
    new_year = year_inc + 1

    pop_mean = multi_feat['World Population'].iloc[starting_index:starting_index+5].mean()
    infl_mean = multi_feat['World Inflation%'].iloc[starting_index:starting_index+5].mean()
    gtax_mean = multi_feat['World Goods Tax%'].iloc[starting_index:starting_index+5].mean()
    wgdp_mean = multi_feat['World GDP%'].iloc[starting_index:starting_index+5].mean()

    df = pd.DataFrame({"Year":[new_year],
                       "World Population":[pop_mean],
                       "World Barrels":0,
                       "World Inflation%":[infl_mean], 
                       "World Goods Tax%":[gtax_mean],
                      "World GDP%":[wgdp_mean]})
    
    multi_feat = multi_feat.append(df, ignore_index=True)
    del df
    
multi_feat

,Year,World Population,World Barrels,World Inflation%,World Goods Tax%,World GDP%
0,2010,6.921872e+09,86855.60512,3.326345,31.875890,24.207113
1,2011,7.002861e+09,87820.24074,4.839403,33.264196,24.547417
2,2012,7.085763e+09,88784.26340,3.707818,33.271756,24.404915
3,2013,7.169640e+09,90151.54476,2.605818,32.787076,24.310278
4,2014,7.254228e+09,90902.90771,2.346269,33.191709,24.470283
5,2015,7.338965e+09,92610.12861,1.393330,33.724915,24.297531
6,2016,7.424282e+09,94403.97687,1.486007,34.248831,23.913640
7,2017,7.509066e+09,96012.59157,2.233522,33.333664,24.222791
8,2018,7.591933e+09,97348.38002,2.458142,34.011405,24.382773
9,2019,7.423695e+09,0.00000,1.983454,33.702105,24.257404


## Multi-step Forecast

In [10]:
future_predict=[]
future_X = multi_feat[["World Population", "World Inflation%", "World Goods Tax%", "World GDP%"]]
future_y = multi_feat["World Barrels"].values.reshape(-1, 1)
X_scaler = StandardScaler().fit(future_X)
# We do not want to include the 0's after 2018. Set range [0:9] (not inclusive).
y_scaler = StandardScaler().fit(future_y[0:9])
X_test_scaled = X_scaler.transform(future_X)
# We do not want to include the 0's after 2018. Set range [0:9] (not inclusive).
y_test_scaled = y_scaler.transform(future_y[0:9])
X_test_scaled_df = pd.DataFrame(X_test_scaled)
y_test_scaled_df = pd.DataFrame(y_test_scaled)

for year in range(5):
    i = 9 + year
    # y_test_scaled_df need to have [year:i+1] to match X_test_scaled_df dimension or it will error with [9,8]
    fitment2 = model.fit(X_test_scaled_df.iloc[year:i], y_test_scaled_df.iloc[year:i+1])
    multi_predict2 = fitment2.predict(X_test_scaled_df.iloc[i-1].values.reshape(1, -1))
    df2 = pd.DataFrame(pd.Series(multi_predict2.flatten()[0]))
    future_predict.append(multi_predict2.flatten()[0])

    y_test_scaled_df = y_test_scaled_df.append(df2, ignore_index=True)
    del df2
    
# Invert future_predict so it's not scaled for later comoparison
inv_future_predict = y_scaler.inverse_transform(future_predict)

print(inv_future_predict)

[97161.84615099 94735.54847339 95289.17910482 95688.38504827
 95856.22239417]


In [11]:
# Create Data Frame for historical and future MSE and R-Square

data = [["Multiple Regression", hist_mse, hist_r2]]

mse_r2_df = pd.DataFrame(data, columns = ["Type", "Historical MSE", "Historical R-Square"])

mse_r2_df

,Type,Historical MSE,Historical R-Square
0,Multiple Regression,0.289547,0.710453


In [12]:
# Export MSE and R-Square summary table as CSV

mse_r2_df.to_csv('../data/clean_data/oil_outputs/OilConsumption_MSE_R2_MultiRegress_Table.csv', index=False)

## Future Prediction

In [13]:
# Generate Prediction table

prediction_19_23 = multi_feat.loc[multi_feat['Year'].between(2019, 2023), ['Year']]
prediction_19_23["Prediction"] = inv_future_predict

prediction_19_23

,Year,Prediction
9,2019,97161.846151
10,2020,94735.548473
11,2021,95289.179105
12,2022,95688.385048
13,2023,95856.222394


## Push Future Predictions to CSV

In [14]:
#Export Future prediction table as CSV

prediction_19_23.to_csv('../data/clean_data/oil_outputs/OilConsumption_Future_MultipleRegression_2019_2023.csv', index=False)